In [228]:
# https://dfrieds.com/data-analysis/rank-method-python-pandas.html
# https://mode.com/blog/bridge-the-gap-window-functions

# 10 Popular SQL Window Functions Replicated in Python

import pandas as pd
import numpy as np
import yfinance as yf
import datetime as dt
import seaborn as sns
%matplotlib inline
color = sns.light_palette("red", as_cmap=True)

orders = pd.read_csv('/Users/antonellobenedetto/Documents/Data_Sets/Medium/customer_orders.csv', parse_dates = ['order_date'] )

orders.columns = ['Order Date','Order ID', 'Customer ID', 'Item ID', 'Item Price', 'Quantity', 'Amount Paid (£)']
orders.insert(1, 'Order Month', pd.DatetimeIndex(orders.index).to_period('M') ) 

orders.head()

#orders.groupby('Customer ID')['Order ID'].count()
#orders.groupby('Customer ID')['Amount Paid (£)'].sum()


,Order Date,Order Month,Order ID,Customer ID,Item ID,Item Price,Quantity,Amount Paid (£)
0,2019-02-13,1970-01,ID001,customer_1,item_3,10.0,2,20.0
1,2019-02-15,1970-01,ID002,customer_2,item_1,23.5,2,47.0
2,2019-02-22,1970-01,ID003,customer_3,item_5,35.0,5,175.0
3,2019-02-16,1970-01,ID004,customer_2,item_2,7.5,3,22.5
4,2019-02-16,1970-01,ID005,customer_1,item_1,23.5,4,94.0


In [229]:
#1. ROW_NUMBER --> RANK(method='first')

orders['Row Num'] = orders.groupby(['Customer ID'])['Order Date'].rank(method='first')
orders.sort_values(by= ['Customer ID','Order Date'], inplace = True)
orders = orders.reset_index(drop = True)

#df.style.background_gradient(cmap=color, subset=pd.IndexSlice[:, ['order_rank_by_client']])
orders.head(5)

,Order Date,Order Month,Order ID,Customer ID,Item ID,Item Price,Quantity,Amount Paid (£),Row Num
0,2019-01-03,1970-01,ID017,customer_1,item_2,7.50,12,90.00,1.0
1,2019-02-03,1970-01,ID018,customer_1,item_1,23.50,10,235.00,2.0
2,2019-02-03,1970-01,ID019,customer_1,item_4,11.75,7,82.25,3.0
3,2019-02-13,1970-01,ID001,customer_1,item_3,10.00,2,20.00,4.0
4,2019-02-16,1970-01,ID005,customer_1,item_1,23.50,4,94.00,5.0


In [230]:
#2. RANK() --> RANK(method='min')
orders['Rank'] = orders.groupby(['Customer ID'])['Order Date'].rank(method='min')
orders.head(5)

,Order Date,Order Month,Order ID,Customer ID,Item ID,Item Price,Quantity,Amount Paid (£),Row Num,Rank
0,2019-01-03,1970-01,ID017,customer_1,item_2,7.50,12,90.00,1.0,1.0
1,2019-02-03,1970-01,ID018,customer_1,item_1,23.50,10,235.00,2.0,2.0
2,2019-02-03,1970-01,ID019,customer_1,item_4,11.75,7,82.25,3.0,2.0
3,2019-02-13,1970-01,ID001,customer_1,item_3,10.00,2,20.00,4.0,4.0
4,2019-02-16,1970-01,ID005,customer_1,item_1,23.50,4,94.00,5.0,5.0


In [231]:
#3. DENSE_RANK() --> RANK(method='dense')
orders['Dense Rank'] = orders.groupby(['Customer ID'])['Order Date'].rank(method='dense')
orders.head(5)

,Order Date,Order Month,Order ID,Customer ID,Item ID,Item Price,Quantity,Amount Paid (£),Row Num,Rank,Dense Rank
0,2019-01-03,1970-01,ID017,customer_1,item_2,7.50,12,90.00,1.0,1.0,1.0
1,2019-02-03,1970-01,ID018,customer_1,item_1,23.50,10,235.00,2.0,2.0,2.0
2,2019-02-03,1970-01,ID019,customer_1,item_4,11.75,7,82.25,3.0,2.0,2.0
3,2019-02-13,1970-01,ID001,customer_1,item_3,10.00,2,20.00,4.0,4.0,3.0
4,2019-02-16,1970-01,ID005,customer_1,item_1,23.50,4,94.00,5.0,5.0,4.0


In [232]:
#4.SUM(...) over(partition by ... order by .. rows unbounded preceding) --> CUMSUM()
orders["Run Tot (£)"] = orders.groupby('Customer ID')['Amount Paid (£)'].cumsum()
orders.head(5)

,Order Date,Order Month,Order ID,Customer ID,Item ID,Item Price,Quantity,Amount Paid (£),Row Num,Rank,Dense Rank,Run Tot (£)
0,2019-01-03,1970-01,ID017,customer_1,item_2,7.50,12,90.00,1.0,1.0,1.0,90.00
1,2019-02-03,1970-01,ID018,customer_1,item_1,23.50,10,235.00,2.0,2.0,2.0,325.00
2,2019-02-03,1970-01,ID019,customer_1,item_4,11.75,7,82.25,3.0,2.0,2.0,407.25
3,2019-02-13,1970-01,ID001,customer_1,item_3,10.00,2,20.00,4.0,4.0,3.0,427.25
4,2019-02-16,1970-01,ID005,customer_1,item_1,23.50,4,94.00,5.0,5.0,4.0,521.25


In [8]:
#5.AVG(...) over(partition by ... order by .. rows unbounded preceding) --> TRANSFORM(np.mean)
orders["Run Avg (£)"] = orders.groupby('Customer ID')['Amount Paid (£)'].transform(np.mean).round(1)
orders.head(5)

,Order Date,Order Month,Order ID,Customer ID,Item ID,Item Price,Quantity,Amount Paid (£),Row Num,Dense Rank,Run Tot (£),Run Avg (£)
0,2019-01-03,1,ID017,customer_1,item_2,7.50,12,90.00,1.0,1.0,90.00,71.8
1,2019-02-03,2,ID018,customer_1,item_1,23.50,10,235.00,2.0,2.0,325.00,71.8
2,2019-02-03,2,ID019,customer_1,item_4,11.75,7,82.25,3.0,2.0,407.25,71.8
3,2019-02-13,2,ID001,customer_1,item_3,10.00,2,20.00,4.0,3.0,427.25,71.8
4,2019-02-16,2,ID005,customer_1,item_1,23.50,4,94.00,5.0,4.0,521.25,71.8


In [110]:
#6 LEAD/LAG (...) over(partition by ... order by ...) --> .SHIFT(n)
aapl = yf.download("AAPL", start="2020-04-15", end="2020-04-22").resample('D').ffill()
#aapl.insert(loc = 0, column = 'Month' ,value = pd.to_datetime(aapl.index).to_period('M') )
aapl.insert(loc = 0, column = 'Symbol' ,value = 'AAPL' )
aapl = aapl[['Symbol', 'Adj Close']].sort_values(by = 'Date', ascending = False)

amzn = yf.download("AMZN", start="2020-04-15", end="2020-04-22").resample('D').ffill()
amzn.insert(loc = 0, column = 'Symbol' ,value = 'AMZN' )
amzn = amzn[['Symbol', 'Adj Close']].sort_values(by = 'Date', ascending = False)

stocks = pd.concat([aapl, amzn], axis = 0)
stocks['Adj Close'] = stocks['Adj Close'].round(2)
#print(stocks.shape)

stocks['Lag1'] = stocks.groupby('Symbol')['Adj Close'].shift(-1)
stocks['Lag1 pct'] = stocks.groupby('Symbol')['Adj Close'].pct_change(-1).round(2)
stocks['Lag7'] = stocks.groupby('Symbol')['Adj Close'].shift(-7)
stocks['Lag7 pct'] = stocks.groupby('Symbol')['Adj Close'].pct_change(-7).round(2)
stocks

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


,Symbol,Adj Close,Lag1,Lag1 pct,Lag7,Lag7 pct
Date,,,,,,
2020-04-21,AAPL,268.37,276.93,-0.03,287.05,-0.07
2020-04-20,AAPL,276.93,282.80,-0.02,NaN,NaN
2020-04-19,AAPL,282.80,282.80,0.00,NaN,NaN
2020-04-18,AAPL,282.80,282.80,0.00,NaN,NaN
2020-04-17,AAPL,282.80,286.69,-0.01,NaN,NaN
2020-04-16,AAPL,286.69,284.43,0.01,NaN,NaN
2020-04-15,AAPL,284.43,287.05,-0.01,NaN,NaN
2020-04-14,AAPL,287.05,NaN,NaN,NaN,NaN
2020-04-21,AMZN,2328.12,2393.61,-0.03,2283.32,0.02


In [262]:
#7. FIRST/LAST VALUE() OVER(partition by ... order by ...) --> rank(method ='first', ascending = True/False) == 1
aapl = yf.download("AAPL", start="2020-03-15", end="2020-04-15").sort_values(by= 'Date', ascending = False).round(1)
aapl.insert(loc = 0, column = 'Month' ,value = pd.to_datetime(aapl.index).to_period('M') )
aapl.reset_index(inplace = True)
aapl = aapl[['Date', 'Month', 'Adj Close']]

aapl['Rank Asc'] = aapl.groupby('Month')['Date'].rank(method ='first')
aapl['Rank Desc'] = aapl.groupby('Month')['Date'].rank(method ='first', ascending = False)

aapl['First Value'] = aapl['Adj Close'].loc[aapl.groupby('Month')['Date'].rank(method ='first') == 1]
aapl['First Value'] = aapl['First Value'].bfill()

aapl['Last Value'] = aapl['Adj Close'].loc[aapl.groupby('Month')['Date'].rank(method ='first', ascending = False) == 1]
aapl['Last Value'] = aapl['Last Value'].ffill()

aapl

[*********************100%***********************]  1 of 1 completed


,Date,Month,Adj Close,Rank Asc,Rank Desc,First Value,Last Value
0,2020-04-14,2020-04,287.0,9.0,1.0,240.9,287.0
1,2020-04-13,2020-04,273.2,8.0,2.0,240.9,287.0
2,2020-04-09,2020-04,268.0,7.0,3.0,240.9,287.0
3,2020-04-08,2020-04,266.1,6.0,4.0,240.9,287.0
4,2020-04-07,2020-04,259.4,5.0,5.0,240.9,287.0
5,2020-04-06,2020-04,262.5,4.0,6.0,240.9,287.0
6,2020-04-03,2020-04,241.4,3.0,7.0,240.9,287.0
7,2020-04-02,2020-04,244.9,2.0,8.0,240.9,287.0
8,2020-04-01,2020-04,240.9,1.0,9.0,240.9,287.0
9,2020-03-31,2020-03,254.3,12.0,1.0,242.2,254.3



With an expanding window, we calculate metrics in an expanding fashion — meaning that we include all rows up to the current one in the calculation. A rolling window allows us to calculate metrics on a rolling basis — for example, rolling(3) means that we use the current observation as well as the two preceding ones in order to calculate our desired metric.

The rationale behind using an expanding window is that with every day that passes, we get another price and another daily change that we can add to our mean calculation. That’s new information that we should capture in our calculated metrics. We can do this with the following code (I also threw in a 3 day rolling window as well for fun).

Calling .expanding() on a pandas dataframe or series creates a pandas expanding object. It’s a lot like the more well known groupby object (which groups things based on specified column labels). The expanding (or rolling) object is what allows us to calculate various metrics in an expanding fashion. Let’s see what our dataframe looks like now:

----
https://stackoverflow.com/questions/45370666/what-are-pandas-expanding-window-functions
To sum up the difference between rolling and expanding function in one line: In rolling function the window size remain constant whereas in the expanding function it changes.
----
https://www.mikulskibartosz.name/the-difference-between-the-expanding-and-rolling-window-in-pandas/
If I use the expanding window with initial size 1, I will create a window that in the first step contains only the first row. In the second step, it contains both the first and the second row. In every step, one additional row is added to the window, and the aggregating function is being recalculated.

In [268]:
#8.AVG(...) over(partition by ... order by .. rows between ... preceding and current row) --> .rolling(window ...).mean()

aapl.sort_values(by= 'Date', ascending = True, inplace = True)
aapl['Rolling Mean 7d'] = aapl['Adj Close'].rolling(7).mean().round(1)
aapl['Expanding Mean 7d'] = aapl['Adj Close'].expanding().mean().round(1)
aapl

,Date,Month,Adj Close,Rank Asc,Rank Desc,First Value,Last Value,Rolling Mean 7d,Expanding Mean 7d
20,2020-03-16,2020-03,242.2,1.0,12.0,242.2,254.3,NaN,242.2
19,2020-03-17,2020-03,252.9,2.0,11.0,242.2,254.3,NaN,247.6
18,2020-03-18,2020-03,246.7,3.0,10.0,242.2,254.3,NaN,247.3
17,2020-03-19,2020-03,244.8,4.0,9.0,242.2,254.3,NaN,246.6
16,2020-03-20,2020-03,229.2,5.0,8.0,242.2,254.3,NaN,243.2
15,2020-03-23,2020-03,224.4,6.0,7.0,242.2,254.3,NaN,240.0
14,2020-03-24,2020-03,246.9,7.0,6.0,242.2,254.3,241.0,241.0
13,2020-03-25,2020-03,245.5,8.0,5.0,242.2,254.3,241.5,241.6
12,2020-03-26,2020-03,258.4,9.0,4.0,242.2,254.3,242.3,243.4
11,2020-03-27,2020-03,247.7,10.0,3.0,242.2,254.3,242.4,243.9
